In [ ]:
!pip install ultralytics opencv-python numba

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import random

from pathlib import Path
from PIL import Image

In [3]:
# data path
train_path = "/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/train/"
upscale_train_path = "/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/"

# 폴더 경로
train_folder = Path(train_path)
upscale_train_folder = Path(upscale_train_path)

# 모든 파일명 가져오기 (파일명만 출력)
only_name = [file.stem for file in train_folder.glob("*.jpg")]
train_seq = [file.name for file in train_folder.iterdir() if file.is_file()]
upscale_train_seq = [file.name for file in upscale_train_folder.iterdir() if file.is_file()]

In [4]:
print(len(train_seq))
print(len(upscale_train_seq))

15834
15834


In [5]:
# test image file path

image_path = train_path + only_name[0] + '.jpg'
upscale_image_path = upscale_train_path + only_name[0] + '.png'

or_image = Image.open(image_path)
up_image = Image.open(upscale_image_path)

print(f"Image size: {or_image.size}")  # (width, height)
print(f"Image size: {up_image.size}")  # (width, height)

# box 구한 뒤 좌표 scale 변경 필요
#   upscale 된 이미지와 일반 이미지의 size 는 정확히 4배 차이
#   updacel 이미지에서 탐지한 객체 박스의 좌표 / 4 ~ 저해상도 상의 좌표
#   4로 안 나눠지는 경우 round / floor / ceil 중에 하나 선택

Image size: (64, 64)
Image size: (256, 256)


##### custom function

In [7]:
# image drawing function

def draw_image(box, image, image_rgb, image_path, resolution_image_path=None):
    x_min, y_min, x_max, y_max = map(int, box)
    bird_crop = image[y_min:y_max, x_min:x_max]  # 크롭된 이미지
    bird_crop_rgb = cv2.cvtColor(bird_crop, cv2.COLOR_BGR2RGB)  # RGB 변환
    bird_resized = cv2.resize(bird_crop_rgb, (64, 64))  # 크기 조정

    # 원본 이미지에 바운딩 박스 표시
    cv2.rectangle(image_rgb, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

    # 결과 비교 시각화
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # 원본 저행상도 이미지
    row_image = Image.open(resolution_image_path)
    axes[0].imshow(row_image)
    axes[0].set_title("Before (Original Image with low resolution)")
    axes[0].axis("off")

    # 원본 이미지 (탐지된 바운딩 박스 포함)
    axes[1].imshow(image_rgb)
    axes[1].set_title("Before (Original Image with Bounding Box)")
    axes[1].axis("off")

    # 크롭된 조류 이미지
    axes[2].imshow(bird_resized)
    axes[2].set_title("After (Cropped & Resized Bird)")
    axes[2].axis("off")

    plt.show()

In [8]:
# 경로에서 파일 이름만 잡아내는 function

def get_filename(s):
    s = s.split("/")
    filename = s[-1]
    filename = filename.split(".")
    return filename[0]

##### yolo detector

In [9]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# YOLO 모델 로드 (사전 학습된 모델)
yolo_model = YOLO("yolov8n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/honey/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 47.9MB/s]


In [50]:
# yolo image detector (면적 기반)

# def bird_detector_yolo(resolution_image_path, model=yolo_model, scale_factor=4):
#     image = cv2.imread(resolution_image_path)
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR이므로 RGB로 변환

#     # 객체 탐지 실행
#     # results = model(image, imgsz=1024) -- 모델 출력 감추기
#     results = model(image, imgsz=1024, verbose=False)

#     # 조류 탐지 후 크롭
#     largest_box = None
#     max_area = 0

#     # 탐지한 bird 객체 중 가장 넓은 면적 find
#     for result in results:
#         for box in result.boxes:
#             cls_id = int(box.cls[0])  # class ID
#             if cls_id == 14:  # 'bird' class ID == 14
#                 x_min, y_min, x_max, y_max = map(int, box.xyxy[0])
#                 area = (x_max - x_min) * (y_max - y_min)  # 박스 면적 계산

#                 # 가장 큰 박스 업데이트
#                 if area > max_area:
#                     max_area = area
#                     largest_box = [x_min, y_min, x_max, y_max]

#     # object box return
#     if largest_box:
#         return [round(point / scale_factor) for point in largest_box]
#     else:
#         return 0

In [10]:
# yolo image detector (신뢰도 기반)

def bird_detector_yolo(resolution_image_path, model=yolo_model, scale_factor=4):
    image = cv2.imread(resolution_image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR이므로 RGB로 변환

    # 객체 탐지 실행
    # results = model(image, imgsz=1024) -- 모델 출력 감추기
    results = model(image, imgsz=1024, verbose=False)

    # 신뢰도가 가장 높은 박스 검출
    highest_confidence = 0
    best_box = None

    # 탐지한 bird 객체 중 가장 넓은 면적 find
    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls[0])  # class ID
            conf = float(box.conf[0])

            if cls_id == 14 and conf > highest_confidence:  # 'bird' class ID == 14
                highest_confidence = conf
                best_box = [int(x) for x in box.xyxy[0]]

    # object box return
    if best_box:
        return best_box, highest_confidence
    else:
        return 0, 0

In [11]:
bird_detector_yolo(upscale_image_path)

([0, 78, 159, 248], 0.2547416090965271)

##### unet detector

In [ ]:
import torch
import torchvision
from torchvision import transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 모델 로드 (SSD with MobileNet V3 backbone)
unet_model = torchvision.models.detection.ssd300_vgg16(pretrained=True)
unet_model.eval()  # 평가 모드로 설정

In [25]:
# unet image detector

def bird_detector_unet(resolution_image_path, model=unet_model, scale_factor=4, conf_score=0.75):
    # 원본 이미지 로드
    image = cv2.imread(resolution_image_path)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR이므로 RGB 변환
    orig_height, orig_width, _ = image.shape  # 원본 이미지 크기 유지

    # 전처리 (Tensor 변환, 이미지 크기 유지)
    transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])
    image_tensor = transform(image_rgb).unsqueeze(0)  # (1, 3, H, W)

    # 모델 예측 실행
    with torch.no_grad():
        outputs = model(image_tensor)

    # 바운딩 박스 및 라벨 추출
    boxes = outputs[0]["boxes"].numpy()
    scores = outputs[0]["scores"].numpy()
    labels = outputs[0]["labels"].numpy()

    # 조류(Bird) 클래스 ID = 15 (COCO 데이터셋 기준)
    bird_boxes = []
    bird_scores = []

    for box, score, label in zip(boxes, scores, labels):
        if label == 16 and score > conf_score:  # 신뢰도 조절 가능
            bird_boxes.append(box)
            bird_scores.append(score)

    if bird_boxes:
        # 신뢰도가 가장 높은 박스 선택
        best_idx = np.argmax(bird_scores)
        best_box = bird_boxes[best_idx]

        return [int(point) for point in best_box], bird_scores[best_idx]
    else:
        return 0, 0


In [14]:
bird_detector_unet(upscale_image_path)

([3, 40, 234, 234], 0.9990416)

##### 객체 탐지 테스트

In [15]:
random.seed(55)

max_num = len(train_seq)
n = 30  # 뽑을 개수

random_numbers = random.sample(range(max_num), n)
print(random_numbers)

[1480, 15575, 3215, 2457, 12113, 4959, 13782, 1305, 12241, 3010, 4946, 1450, 5761, 15007, 11826, 15111, 15821, 14197, 10856, 7732, 8856, 6313, 6355, 8249, 6647, 5213, 11781, 15603, 7026, 8752]


In [ ]:
# for i in random_numbers:
#     # image_path = train_path + only_name[i] + '.jpg'
#     upscale_image_path = upscale_train_path + only_name[i] + '.png'

#     print(i, " 번째 이미지 객체 탐지 결과")
#     print("yolo model - ", bird_detector_yolo(upscale_image_path))
#     print("unet model - ", bird_detector_unet(upscale_image_path))
#     print("\n")

##### master dataframe

In [33]:
# train dataframe import

train_df = pd.read_csv("/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/train.csv")
train_df["img_filename"] = train_df["upscale_img_path"].apply(lambda x: get_filename(x))

In [34]:
train_df.head()

,img_path,upscale_img_path,label,img_filename
0,./train/TRAIN_00000.jpg,./upscale_train/TRAIN_00000.png,Ruddy Shelduck,TRAIN_00000
1,./train/TRAIN_00001.jpg,./upscale_train/TRAIN_00001.png,Gray Wagtail,TRAIN_00001
2,./train/TRAIN_00002.jpg,./upscale_train/TRAIN_00002.png,Indian Peacock,TRAIN_00002
3,./train/TRAIN_00003.jpg,./upscale_train/TRAIN_00003.png,Common Kingfisher,TRAIN_00003
4,./train/TRAIN_00004.jpg,./upscale_train/TRAIN_00004.png,Common Kingfisher,TRAIN_00004


In [35]:
# 새로운 칼럼 추가
train_df["yolo_box"] = None
train_df["yolo_conf"] = None
train_df["unet_box"] = None
train_df["unet_conf"] = None

In [36]:
# 각 행을 순회하면서 YOLO 및 U-Net 모델 실행
for index, row in train_df.iterrows():
    upscale_image = row["img_filename"]  # 이미지 파일명
    upscale_image_path = upscale_train_path + upscale_image + '.png'
    print(upscale_image_path)

    # YOLO 모델 실행
    yolo_result = bird_detector_yolo(upscale_image_path)  # [box, conf] 형태
    if isinstance(yolo_result, tuple):  # (bbox, confidence) 형식이면 저장
        yolo_box, yolo_conf = yolo_result
    else:
        yolo_box, yolo_conf = None, None  # 객체 감지 실패 시

    # U-Net 모델 실행
    unet_result = bird_detector_unet(upscale_image_path)  # [box, conf] 형태
    if isinstance(unet_result, tuple):
        unet_box, unet_conf = unet_result
    else:
        unet_box, unet_conf = None, None

    # 데이터프레임에 결과 저장
    train_df.at[index, "yolo_box"] = yolo_box
    train_df.at[index, "yolo_conf"] = yolo_conf
    train_df.at[index, "unet_box"] = unet_box
    train_df.at[index, "unet_conf"] = unet_conf

    if (index % 100) == 0:
        print(index, " 번째 이미지 작업 완료")


/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00000.png
0  번째 이미지 작업 완료
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00001.png
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00002.png
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00003.png
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00004.png
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00005.png
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00006.png
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00007.png
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_00008.png
/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/data/open/upscale_train/TRAIN_0000

In [37]:
train_df.to_csv("/Users/honey/Desktop/2025-1 KUBIG/25 Winter/DL study/팀프로젝트/master_train.csv")